In [1]:
import os
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Markdown
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
import dotenv
from dotenv import load_dotenv
from pathlib import Path
import pprint

Load model

In [2]:
load_dotenv()

# Access the Groq API key from environment variables
groq_api_key = os.getenv('GROQ_API_KEY')

In [3]:
model = 'llama-3.3-70b-versatile'

def load_model(model, temperature):
    llm = ChatGroq(
        model=model,
        temperature=temperature,
        max_tokens=None,
        timeout=None,
        max_retries=2
    )
    return llm

In [4]:
llm = load_model(model, temperature=0.7)

In [5]:
prompt = 'How can I reset my password in AWS?'

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful customer support assistant.'),
    ('human', '{prompt}'),
    
])

# Create chain to combine prompt and model and get response
chain = template | llm | StrOutputParser()
res = chain.invoke(
      {'prompt': prompt}
  )
print(res)

Resetting your password in AWS (Amazon Web Services) can be done in a few simple steps. Here's a step-by-step guide to help you reset your password:

**Method 1: Reset Password using the AWS Management Console**

1. Go to the AWS Management Console sign-in page: https://console.aws.amazon.com/
2. Click on "Forgot password" below the sign-in button.
3. Enter your AWS account email address or account ID, and then click "Next".
4. Enter the verification code sent to your email address or phone number (if you have 2-factor authentication enabled).
5. Create a new password and confirm it by re-entering it in the next field.
6. Click "Save changes" to update your password.

**Method 2: Reset Password using the AWS CLI**

1. Install and configure the AWS CLI on your machine.
2. Run the command `aws iam change-password --old-password <old_password> --new-password <new_password>` (replace `<old_password>` and `<new_password>` with your actual passwords).
3. If you're using an IAM user, you'll n

In [6]:
def show_response(response):
    
    if '</think>' in response: # Check if the response contains the </think> tag
        response = response.split('</think>')[-1].strip() # Extract content after the </think> tag
    
    else:
        response = response.strip() # No </think> tag, use the whole response
    
    display(Markdown(response))
    
show_response(res)

Resetting your password in AWS (Amazon Web Services) can be done in a few simple steps. Here's a step-by-step guide to help you reset your password:

**Method 1: Reset Password using the AWS Management Console**

1. Go to the AWS Management Console sign-in page: https://console.aws.amazon.com/
2. Click on "Forgot password" below the sign-in button.
3. Enter your AWS account email address or account ID, and then click "Next".
4. Enter the verification code sent to your email address or phone number (if you have 2-factor authentication enabled).
5. Create a new password and confirm it by re-entering it in the next field.
6. Click "Save changes" to update your password.

**Method 2: Reset Password using the AWS CLI**

1. Install and configure the AWS CLI on your machine.
2. Run the command `aws iam change-password --old-password <old_password> --new-password <new_password>` (replace `<old_password>` and `<new_password>` with your actual passwords).
3. If you're using an IAM user, you'll need to specify the `--username` option followed by your IAM username.

**Method 3: Reset Password using the AWS IAM Dashboard**

1. Sign in to the AWS Management Console using your IAM user credentials.
2. Navigate to the IAM dashboard: https://console.aws.amazon.com/iam/
3. Click on your username in the top right corner, then select "Security credentials".
4. Click on "Password" and then click on "Update".
5. Enter your old password, new password, and confirm your new password.
6. Click "Update password" to save the changes.

Remember to choose a strong and unique password for your AWS account, and consider enabling 2-factor authentication for added security.

If you're still having trouble resetting your password, feel free to provide more details about your issue, and I'll be happy to help you further.

Define context

In [7]:
context = """"
To change your AWS password, follow these steps:
1. Go to the AWS Management Console and log in with your current credentials.
2. Click on your account name in the top right corner and select "My Security
Credentials" from the dropdown menu.
3. In the "Password" section, click on the "Change Password" button.
4. Enter your current password, then enter and confirm your new password.
5. Click "Save Changes" to update your password.
6. You will receive a confirmation message indicating that your password has been successfully changed.
"""

prompt = f"""
How can I reset my password in AWS?
Here is some context that might help you:

Context: {context}
Please provide step-by-step instructions.
"""
res = chain.invoke(
      {'prompt': prompt}
  )

show_response(res)

To reset your password in AWS, follow these step-by-step instructions:

**Step 1: Log in to the AWS Management Console**
Go to the AWS Management Console and log in with your current credentials.

**Step 2: Access Your Security Credentials**
Click on your account name in the top right corner of the page, and then select "My Security Credentials" from the dropdown menu.

**Step 3: Navigate to the Password Section**
In the "My Security Credentials" page, locate the "Password" section.

**Step 4: Initiate the Password Change**
Click on the "Change Password" button in the "Password" section.

**Step 5: Update Your Password**
Enter your current password, then enter and confirm your new password. Make sure to choose a strong and unique password.

**Step 6: Save Your Changes**
Click "Save Changes" to update your password.

**Step 7: Verify the Change**
You will receive a confirmation message indicating that your password has been successfully changed.

That's it! Your AWS password should now be reset. If you have any issues or concerns, feel free to ask.

Prompt For RAG

In [8]:
template_rag = """
Question: {input}
Context: {context}
"""

In [9]:
prompt_rag = PromptTemplate.from_template(template_rag)

print(prompt_rag)

input_variables=['context', 'input'] template='\nQuestion: {input}\nContext: {context}\n'


Create Chain 

In [10]:
chain_rag = prompt_rag | llm | StrOutputParser() # Create RAG chain to combine prompt, context and model so as to get response

input_question = "How can I reset my password in AWS?"

rag_response = chain_rag.invoke({
    'context': context,
    'input': input_question
})
show_response(rag_response)

To reset your password in AWS, you can follow these steps:

1. Log in to the AWS Management Console with your current credentials.
2. Click on your account name in the top right corner and select "My Security Credentials".
3. In the "Password" section, click on the "Change Password" button.
4. Enter your current password, then enter and confirm your new password.
5. Click "Save Changes" to update your password.

Note: The provided context does not explicitly mention "resetting" a password, but rather "changing" a password. If you have forgotten your password, the process may be different. However, based on the given context, the above steps can be used to change your AWS password. If you are unable to log in due to a forgotten password, you may need to contact AWS support for further assistance.

Indexing Steps
* Upload and prepare documents

In [11]:

file_path = r'C:\Users\USER\LLMOPs_Projects\customer_support\data\safebank-manual.pdf'

# loader = PyPDFLoader(file_path)
# documents = loader.load()  
# print(f'Number of pages in document: {len(documents)}')
# for i, doc in enumerate(documents):
#     print(f'--- Page {i+1} ---')
#     print(doc.page_content)
#     print()

if os.path.isfile(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()  
    print(f'Number of pages in document: {len(documents)}')
    for i, doc in enumerate(documents):
        print(f'--- Page {i+1} ---')
        print(doc.page_content)
        print()


Number of pages in document: 10
--- Page 1 ---
Customer
 
Service
 
and
 
Usage
 
Manual
 
–
 
SafeBank
 
 
1.
 
Introduction
 
Goal
 
of
 
the
 
Manual
 
This
 
manual
 
aims
 
to
 
guide
 
both
 
customers
 
and
 
support
 
agents
 
through
 
the
 
full
 
functionality
 
of
 
the
 
digital
 
bank,
 
covering
 
everything
 
from
 
account
 
access,
 
features,
 
and
 
services
 
to
 
security
 
procedures,
 
support,
 
and
 
problem
 
resolution.
 
It
 
also
 
serves
 
as
 
a
 
reference
 
material
 
for
 
frequently
 
asked
 
questions.
 
Overview
 
of
 
the
 
Digital
 
Bank
 
Our
 
digital
 
bank
 
provides
 
a
 
full
 
range
 
of
 
banking
 
services
 
entirely
 
online,
 
through
 
both
 
the
 
app
 
and
 
website.
 
Key
 
services
 
include
 
free
 
digital
 
accounts,
 
bank
 
transfers,
 
bill
 
payments,
 
card
 
issuance,
 
investments,
 
insurance,
 
and
 
more
 
—
 
always
 
with
 
a
 
focus
 
on
 
convenience,
 
security,
 
and
 
transparency.
 
 
Target
 
Audience
 
The
 

In [12]:
loader = PyMuPDFLoader(file_path)
docs = loader.load()
print(f'Number of pages in document: {len(docs)}')

Number of pages in document: 10


In [13]:
docs

[Document(metadata={'source': 'C:\\Users\\USER\\LLMOPs_Projects\\customer_support\\data\\safebank-manual.pdf', 'file_path': 'C:\\Users\\USER\\LLMOPs_Projects\\customer_support\\data\\safebank-manual.pdf', 'page': 0, 'total_pages': 10, 'format': 'PDF 1.4', 'title': 'safebank-manual', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Skia/PDF m140 Google Docs Renderer', 'creationDate': '', 'modDate': '', 'trapped': ''}, page_content='Customer Service and Usage Manual – SafeBank \n \n1. Introduction \nGoal of the Manual \nThis manual aims to guide both customers and support agents through the full functionality \nof the digital bank, covering everything from account access, features, and services to \nsecurity procedures, support, and problem resolution. It also serves as a reference material \nfor frequently asked questions. \nOverview of the Digital Bank \nOur digital bank provides a full range of banking services entirely online, through both the \napp and websit

In [14]:
print(docs[0])

page_content='Customer Service and Usage Manual – SafeBank 
 
1. Introduction 
Goal of the Manual 
This manual aims to guide both customers and support agents through the full functionality 
of the digital bank, covering everything from account access, features, and services to 
security procedures, support, and problem resolution. It also serves as a reference material 
for frequently asked questions. 
Overview of the Digital Bank 
Our digital bank provides a full range of banking services entirely online, through both the 
app and website. Key services include free digital accounts, bank transfers, bill payments, 
card issuance, investments, insurance, and more — always with a focus on convenience, 
security, and transparency.  
Target Audience 
The digital bank is aimed at individuals aged 18 and over who are seeking modern, 
accessible financial solutions with fully digital management.  
 
2. Registration and Account Opening 
Requirements to Open an Account 
●​ Be at least 18 years

In [15]:
pprint.pp(docs[0].page_content)

('Customer Service and Usage Manual – SafeBank \n'
 ' \n'
 '1. Introduction \n'
 'Goal of the Manual \n'
 'This manual aims to guide both customers and support agents through the full '
 'functionality \n'
 'of the digital bank, covering everything from account access, features, and '
 'services to \n'
 'security procedures, support, and problem resolution. It also serves as a '
 'reference material \n'
 'for frequently asked questions. \n'
 'Overview of the Digital Bank \n'
 'Our digital bank provides a full range of banking services entirely online, '
 'through both the \n'
 'app and website. Key services include free digital accounts, bank transfers, '
 'bill payments, \n'
 'card issuance, investments, insurance, and more — always with a focus on '
 'convenience, \n'
 'security, and transparency.  \n'
 'Target Audience \n'
 'The digital bank is aimed at individuals aged 18 and over who are seeking '
 'modern, \n'
 'accessible financial solutions with fully digital management.  \n'
 

In [16]:
len(docs[0].page_content)

1589

In [17]:
docs[0].page_content[:100]

'Customer Service and Usage Manual – SafeBank \n \n1. Introduction \nGoal of the Manual \nThis manual aim'

In [18]:
print()
pprint.pp(docs[0].metadata)


{'source': 'C:\\Users\\USER\\LLMOPs_Projects\\customer_support\\data\\safebank-manual.pdf',
 'file_path': 'C:\\Users\\USER\\LLMOPs_Projects\\customer_support\\data\\safebank-manual.pdf',
 'page': 0,
 'total_pages': 10,
 'format': 'PDF 1.4',
 'title': 'safebank-manual',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Skia/PDF m140 Google Docs Renderer',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}


In [19]:
# Define a function to extract text from documents
def extract_text_from_docs(file_path):
    loader = PyMuPDFLoader(file_path)
    docs = loader.load()
    content = '\n'.join([page.page_content for page in docs])
    
    return content

In [20]:
loaded_docs = [extract_text_from_docs(pdf) for pdf in Path(r'C:\Users\USER\LLMOPs_Projects\customer_support\data').glob('*.pdf')]
len(loaded_docs)

1

In [21]:
loaded_docs

['Customer Service and Usage Manual – SafeBank \n \n1. Introduction \nGoal of the Manual \nThis manual aims to guide both customers and support agents through the full functionality \nof the digital bank, covering everything from account access, features, and services to \nsecurity procedures, support, and problem resolution. It also serves as a reference material \nfor frequently asked questions. \nOverview of the Digital Bank \nOur digital bank provides a full range of banking services entirely online, through both the \napp and website. Key services include free digital accounts, bank transfers, bill payments, \ncard issuance, investments, insurance, and more — always with a focus on convenience, \nsecurity, and transparency.  \nTarget Audience \nThe digital bank is aimed at individuals aged 18 and over who are seeking modern, \naccessible financial solutions with fully digital management.  \n \n2. Registration and Account Opening \nRequirements to Open an Account \n●\u200b Be at le

In [22]:
print(extract_text_from_docs(file_path))

Customer Service and Usage Manual – SafeBank 
 
1. Introduction 
Goal of the Manual 
This manual aims to guide both customers and support agents through the full functionality 
of the digital bank, covering everything from account access, features, and services to 
security procedures, support, and problem resolution. It also serves as a reference material 
for frequently asked questions. 
Overview of the Digital Bank 
Our digital bank provides a full range of banking services entirely online, through both the 
app and website. Key services include free digital accounts, bank transfers, bill payments, 
card issuance, investments, insurance, and more — always with a focus on convenience, 
security, and transparency.  
Target Audience 
The digital bank is aimed at individuals aged 18 and over who are seeking modern, 
accessible financial solutions with fully digital management.  
 
2. Registration and Account Opening 
Requirements to Open an Account 
●​ Be at least 18 years old 
●​ Have 

Split document file into chunks

In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = []

for doc in loaded_docs:
    chunks.extend(text_splitter.split_text(doc)) # Split each document into chunks and add to the list
print(f'Total Number of Chunk created: {len(chunks)}')


Total Number of Chunk created: 44


In [24]:
chunks[0]

'Customer Service and Usage Manual – SafeBank \n \n1. Introduction \nGoal of the Manual \nThis manual aims to guide both customers and support agents through the full functionality \nof the digital bank, covering everything from account access, features, and services to \nsecurity procedures, support, and problem resolution. It also serves as a reference material \nfor frequently asked questions. \nOverview of the Digital Bank'

Embedding: Converting text into vector

In [25]:
embedding_model = 'BAAI/bge-large-en-v1.5'
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

input = 'How are you?'

result = embeddings.embed_query(input)

c:\Users\USER\LLMOPs_Projects\customer_support\customer-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
result[:10]

[0.05450529232621193,
 -0.023465581238269806,
 -0.06775867938995361,
 0.039945267140865326,
 -0.028649508953094482,
 -0.0655255913734436,
 0.028446827083826065,
 0.05666505917906761,
 0.02517046593129635,
 0.006748091895133257]

Storing the vectors into vector database

In [27]:
vectorestore = FAISS.from_texts(chunks, embedding=embeddings)

#Save the Faiss index
vectorestore.save_local('faiss_index')


In [28]:
vectorestore

Retrieval and Generation
* Configuring the Retriever

In [29]:
retriever = vectorestore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 6}
)
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A29B131BD0>, search_kwargs={'k': 6})

In [30]:
# Generation

prompt_rag = PromptTemplate(
    input_variables=['context', 'input'],
    template=template_rag
)

prompt_rag

PromptTemplate(input_variables=['context', 'input'], template='\nQuestion: {input}\nContext: {context}\n')

In [31]:
chain_rag = (
    {
        'context': retriever,
        'input': RunnablePassthrough()
    }
    | prompt_rag
    | llm
    | StrOutputParser()
)

In [32]:
text = 'How can i change my password?'
res = chain_rag.invoke(text)
show_response(res)

To change your password, you can follow these steps:

**Through the App:**
1. Go to "My Account"
2. Select "Change Password"
3. Enter your current password
4. Choose a new password (note: recently used passwords cannot be reused)

**Through the Website:**
1. Navigate to "Settings"
2. Select "My Account"
3. Click on "Change Password"
4. Enter your current password
5. Choose a new password (note: recently used passwords cannot be reused)

If you forget your login credentials, you can select "Forgot Password" on the login screen and follow the instructions sent via email or SMS to set a new password.

In [33]:
# Improved prompt

system_prompt = """
You are a helpful virtual assistant anwering general question about company's service.
Use the following bits of retrieved context to answer the question.
If you don't know the answer, tell me you doon't know. Keep your answer concise. \n\n
"""

qa_prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', 'Question: {input}\n\n Context: {context}'),
])

qa_prompt

ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are a helpful virtual assistant anwering general question about company's service.\nUse the following bits of retrieved context to answer the question.\nIf you don't know the answer, tell me you doon't know. Keep your answer concise. \n\n\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='Question: {input}\n\n Context: {context}'))])

In [34]:
chain_rag = (
    {
        'context': retriever,
        'input': RunnablePassthrough()
    }
    | qa_prompt
    | llm
    |StrOutputParser()
)

text = 'How can i change my password?'
res = chain_rag.invoke(text)
show_response(res)

To change your password, go to "My Account" > "Change Password" in the app, or "Settings" > "My Account" > "Change Password" on the website. You'll need to enter your current password, and you can't reuse a recently used password. Alternatively, you can also select "Forgot Password" on the login screen and follow the instructions sent via email or SMS.

Advanced RAG Pipeline

In [35]:
# Query reformulation to contextualiza

context_q_system_prompt = "Given the following chat history and the following question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do not answer the question, just reformulate it if needed and otherwise return it as is."

context_q_system_prompt = context_q_system_prompt
context_q_user_prompt = 'Question: {input}'
context_q_prompt = ChatPromptTemplate.from_messages([
    ('system', context_q_system_prompt),
    MessagesPlaceholder('chat_history'),
    ('human', context_q_user_prompt)
])

In [36]:
# Chain for contextualization

history_aware_retriever = create_history_aware_retriever(
    llm= llm,
    retriever= retriever,
    prompt= context_q_prompt
)

In [37]:
# Question and answer chain (Q and A)
qa_prompt = ChatPromptTemplate.from_messages([
     ('system', system_prompt),
    MessagesPlaceholder('chat_history'),
    ('human', 'Question: {input}\n\n Context: {context}')
])

qa_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

In [38]:
def config_rag_chain(llm, retriever):
    
    # Contextualization prompt
    context_q_system_prompt = "Given the following chat history and the following question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do not answer the question, just reformulate it if needed and otherwise return it as is."

    context_q_system_prompt = context_q_system_prompt
    context_q_user_prompt = 'Question: {input}'
    context_q_prompt = ChatPromptTemplate.from_messages([
        ('system', context_q_system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', context_q_user_prompt)
    ])
    
    # Chain for contextualization

    history_aware_retriever = create_history_aware_retriever(
        llm= llm,
        retriever= retriever,
        prompt= context_q_prompt
    )
    
    #Question and Answer Prompt
    system_prompt = """
    You are a helpful virtual assistant anwering general question about company's service.
    Use the following bits of retrieved context to answer the question.
    If you don't know the answer, tell me you doon't know. Keep your answer concise. \n\n
    """
        
    # Question and answer chain (Q and A)
    qa_prompt = ChatPromptTemplate.from_messages([
        ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', 'Question: {input}\n\n Context: {context}')
    ])
    
    system_prompt = """. \n\n"""

    qa_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(
        history_aware_retriever,
        qa_chain
    )
    
    return rag_chain

Create chat history

In [39]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [40]:
qa_chain_with_history = RunnableWithMessageHistory(
    qa_chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history'
)

In [41]:
session_id = 'user_42'

In [42]:
response1 = qa_chain_with_history.invoke(
    {'input': 'How can i change my password in the app?',
     'context': ''
    },
    config = {'configurable': {'session_id': session_id}                               }
)
print(response1)

I don't know. There's no context provided about the app or its password change process.


In [43]:
response2 = qa_chain_with_history.invoke(
    {'input': 'What was my last question?',
     'context': ''
    },
    config = {'configurable': {'session_id': session_id}                               }
)
print(response2)

Your last question was "How can i change my password in the app?"


Add chat history to our chain

In [44]:
rag_chain = config_rag_chain(llm, retriever)

In [45]:
chat_history = []

input = 'How to change password?'

chat_history.append(HumanMessage(content=input))

result = rag_chain.invoke({
    'input': input,
    'chat_history': chat_history
})

response = result['answer']
print(result)

chat_history.append(AIMessage(content=response))

{'input': 'How to change password?', 'chat_history': [HumanMessage(content='How to change password?')], 'context': [Document(page_content='additional verification will be required. Users can view and remove authorized devices \nthrough the app or website. \nPassword Change\u200b\nTo change your password in the app, go to "My Account" > "Change Password." On the \nwebsite, navigate to "Settings" > "My Account" > "Change Password." The current password \nmust be entered, and recently used passwords cannot be reused. \nAccess Recovery\u200b\nIf you forget your login credentials, select "Forgot Password" on the login screen. The'), Document(page_content='devices. \nPassword and Login Recovery \nClick "Forgot my password" on the login screen. Enter your ID number and follow the \ninstructions sent via email or SMS to set a new password. If you don’t receive the code, \ncheck your spam folder or contact support. \nBiometric and Facial Recognition Setup  \nIn the app, go to "Settings > Securi

In [46]:
chat_history

[HumanMessage(content='How to change password?'),
 AIMessage(content='To change your password, go to "My Account" > "Change Password" in the app or "Settings" > "My Account" > "Change Password" on the website. You\'ll need to enter your current password, and you can\'t reuse recently used passwords.')]

In [ ]:
input = 